In [1]:
import tensorflow as tf
import numpy as np
import os
import sys
import data_helpers
from tensorflow.contrib import learn
from shallowCNN import TextCNN

/home/yuji/anaconda3/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
# Model Initialization
tf.flags.DEFINE_string("model_path", "../runs/1512298518/checkpoints/model-100", "Path to the trained model.")

# Data loading params
tf.flags.DEFINE_float("dev_sample_percentage", .1, "Percentage of the training data to use for validation")
tf.flags.DEFINE_string("positive_data_file", "../yelp_sentences.txt", "Data source for the positive data.")
tf.flags.DEFINE_string("negative_data_file", "../runner_up.txt", "Data source for the negative data.")

# Model Hyperparameters
tf.flags.DEFINE_integer("embedding_dim", 14351, "Dimensionality of character embedding (default: 128)")
tf.flags.DEFINE_string("filter_sizes", "3,4,5", "Comma-separated filter sizes (default: '3,4,5')")
tf.flags.DEFINE_integer("num_filters", 128, "Number of filters per filter size (default: 128)")
tf.flags.DEFINE_float("dropout_keep_prob", 0.5, "Dropout keep probability (default: 0.5)")
tf.flags.DEFINE_float("l2_reg_lambda", 0.0, "L2 regularization lambda (default: 0.0)")
tf.flags.DEFINE_integer("num_reconstructions", 1, "Number of reconstructions to built (default: 1)")

FLAGS = tf.flags.FLAGS
FLAGS._parse_flags()
print("Parameters:")
for attr, value in sorted(FLAGS.__flags.items()):
    print("{}={}".format(attr.upper(), value))

Parameters:
DEV_SAMPLE_PERCENTAGE=0.1
DROPOUT_KEEP_PROB=0.5
EMBEDDING_DIM=14351
FILTER_SIZES=3,4,5
L2_REG_LAMBDA=0.0
MODEL_PATH=../runs/1512298518/checkpoints/model-100
NEGATIVE_DATA_FILE=../runner_up.txt
NUM_FILTERS=128
NUM_RECONSTRUCTIONS=1
POSITIVE_DATA_FILE=../yelp_sentences.txt


In [3]:
# Load data
print("Loading data...")
x_text, y = data_helpers.load_data_and_labels(FLAGS.positive_data_file, FLAGS.negative_data_file)

# Build vocabulary
max_document_length = max([len(x.split(" ")) for x in x_text])
vocab_processor = learn.preprocessing.VocabularyProcessor(max_document_length)
x = np.array(list(vocab_processor.fit_transform(x_text)))

print("Vocabulary Size: {:d}".format(len(vocab_processor.vocabulary_)))

Loading data...
Vocabulary Size: 14351


In [4]:
def onehotencode(vector, num_classes=None):
    """
    Converts an input 1-D vector of integers into an output
    2-D array of one-hot vectors, where an i'th input value
    of j will set a '1' in the i'th row, j'th column of the
    output array.
    """
    assert isinstance(vector, np.ndarray)
    assert len(vector) > 0

    if num_classes is None:
        num_classes = np.max(vector)+1
    else:
        assert num_classes > 0
        assert num_classes >= np.max(vector)

    result = np.zeros(shape=(len(vector), num_classes))
    result[np.arange(len(vector)), vector] = 1
    return result.astype(int)

In [5]:
with tf.Graph().as_default():
    session_conf = tf.ConfigProto(allow_soft_placement=True, log_device_placement=False)
    sess = tf.Session(config=session_conf)
    
    with sess.as_default() as sess:
        if FLAGS.model_path in (None, ""):
            raise ValueError("`model_path` must be specified, but was:", FLAGS.model_path)

        print("Loading saved model")
        restore_saver = tf.train.import_meta_graph(FLAGS.model_path + ".meta")  # Construct the graph
        restore_saver.restore(sess, FLAGS.model_path)  # Initialize the variables
        print("Constructing Reconstruction Graph...")
        
        cnn = TextCNN(
            sequence_length=x.shape[1],
            num_classes=y.shape[1],
            vocab_size=len(vocab_processor.vocabulary_),
            embedding_size=FLAGS.embedding_dim,
            filter_sizes=list(map(int, FLAGS.filter_sizes.split(","))),
            num_filters=FLAGS.num_filters,
            l2_reg_lambda=FLAGS.l2_reg_lambda)

        # Expand data into the embeddings
        data_batch = x[:1]
        vocab_size = len(vocab_processor.vocabulary_)
        embeddings_batch = np.zeros(data_batch.shape + (FLAGS.embedding_dim,))
        for i, sentence in enumerate(data_batch):
            embeddings_batch[i] = onehotencode(sentence, num_classes=vocab_size)
        embeddings_batch = embeddings_batch.reshape(embeddings_batch.shape + (1,))

        # Assign reconstruction tensor to data_batch to generate target_content
        target_content = sess.run(cnn.conv, feed_dict={cnn.embedded_chars_expanded: embeddings_batch})

Loading saved model
INFO:tensorflow:Restoring parameters from ../runs/1512298518/checkpoints/model-100
Constructing Reconstruction Graph...


FailedPreconditionError: Attempting to use uninitialized value conv-maxpool-5/W_1
	 [[Node: conv-maxpool-5/W_1/read = Identity[T=DT_FLOAT, _class=["loc:@conv-maxpool-5/W_1"], _device="/job:localhost/replica:0/task:0/device:CPU:0"](conv-maxpool-5/W_1)]]

Caused by op 'conv-maxpool-5/W_1/read', defined at:
  File "/home/yuji/anaconda3/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/yuji/anaconda3/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/yuji/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/yuji/anaconda3/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/home/yuji/anaconda3/lib/python3.6/site-packages/ipykernel/kernelapp.py", line 477, in start
    ioloop.IOLoop.instance().start()
  File "/home/yuji/anaconda3/lib/python3.6/site-packages/zmq/eventloop/